In [28]:
# ライブラリのインポート
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [29]:
# ページ数を取得
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2="
res = requests.get(base_url)

# ステータスコードが正常なら実行
if res.status_code == 200: 

    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    page_elements = soup.select("ol.pagination-parts a")
    page_numbers = [int(element.text) for element in page_elements] # aタグ内に記載されたページ数をリスト化
    total_pages = max(page_numbers)

print("ページ数：" + str(total_pages))

ページ数：3


In [30]:
# データフレームを作成
columns = ["物件名","住所","間取り","階数","家賃","管理費","敷金","礼金","専有面積","築年数"]
df = pd.DataFrame(columns=columns)

df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数


In [31]:
# ステータスコードが正常なら実行
if res.status_code == 200: 
    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    # 各ページにアクセスして情報を取得
    for page_number in range(1, total_pages + 1):
        page_url = template_url.format(page_number)
        page_response = requests.get(page_url)

        # 必要な情報を抽出
        property_items = soup.select('.cassetteitem')

        for item in property_items:
            title = item.select_one('.cassetteitem_content-title').text  # 物件名の取得
            address = item.select_one('.cassetteitem_detail-col1').text  # 住所の取得
            age = item.select_one('.cassetteitem_detail-col3 div').text  # 築年数の取得
            
            floor_level = item.select_one('.js-cassette_link td:nth-of-type(3)').text.strip()  # 階数の取得
            rent_fee = item.select_one('.cassetteitem_price--rent .ui-text--bold').text  # 家賃の取得
            management_fee = item.select_one('.cassetteitem_price--administration').text  # 管理費の取得
            deposit = item.select_one('.cassetteitem_price--deposit').text  # 敷金の取得
            gratuity_fee = item.select_one('.cassetteitem_price--gratuity').text  # 礼金の取得
            occupied_area = item.select_one('.cassetteitem_menseki').text  # 専有面積の取得
            floor_plan = item.select_one('.cassetteitem_madori').text  # 間取りの取得

            # 抽出した情報をデータフレームに追加
            add_data = {
                "物件名": [title],
                "住所": [address],
                "間取り": [floor_plan],
                "階数": [floor_level],
                "家賃": [rent_fee],
                "管理費": [management_fee],
                "敷金": [deposit],
                "礼金": [gratuity_fee],
                "専有面積": [occupied_area],
                "築年数": [age]
            }

            add_df = pd.DataFrame(add_data)
            df = pd.concat([df, add_df], ignore_index=True)

In [32]:
df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数
0,菅野ビル,東京都目黒区中根１,2LDK,4階,19.5万円,5000円,19.5万円,-,74.77m2,築34年
1,ストークハイツ中島,東京都目黒区上目黒３,3LDK,2階,19万円,-,19万円,19万円,60.3m2,築33年
2,東建マンション学芸大,東京都目黒区中央町２,3LDK,4階,15.5万円,6000円,15.5万円,15.5万円,49.5m2,築51年
3,東急東横線 学芸大学駅 8階建 築51年,東京都目黒区中央町２,3LDK,4階,15.5万円,6000円,15.5万円,15.5万円,49.5m2,築51年
4,Green Forest Gohongi,東京都目黒区五本木３,2LDK,1階,18万円,3000円,-,18万円,58.03m2,築3年
...,...,...,...,...,...,...,...,...,...,...
85,京王井の頭線 駒場東大前駅 5階建 築8年,東京都目黒区駒場３,2LDK,1階,18.3万円,15000円,18.3万円,18.3万円,47.39m2,築8年
86,東急東横線 学芸大学駅 5階建 新築,東京都目黒区鷹番３,2LDK,1階,18.6万円,12000円,18.6万円,18.6万円,40.8m2,新築
87,アーバンフラッツ目黒本町,東京都目黒区目黒本町４,2LDK,1階,18万円,10000円,18万円,18万円,58.32m2,築18年
88,東急東横線 祐天寺駅 4階建 築28年,東京都目黒区祐天寺１,2LDK,3階,20万円,5000円,20万円,20万円,47.94m2,築28年
